In [2]:
import time
start=time.time()
class pattern:
    def __init__(self, iset, s):
        self.itemset = iset
        self.support = s


class TreeNode:
    def __init__(self, item, support):
        self.itemset = item
        self.support = support
        self.children = []
        self.parent = None

class FPTree:
    def __init__(self):
        self.root = TreeNode(None, 0)
        self.header_table = {}

    def insert(self, transaction):
        current_node = self.root

        for item in transaction:
            found = False
            for child_node in current_node.children:
                if item == child_node.itemset:
                    found = True
                    current_node = child_node
                    current_node.support += 1
                    break

            if not found:
                new_node = TreeNode(item, 1)
                new_node.parent = current_node
                current_node.children.append(new_node)
                current_node = new_node
                if item in self.header_table:
                    self.header_table[item].append(new_node)
                else:
                    self.header_table[item] = [new_node]

    def print_tree(self):
        self._print_tree(self.root)

    def _print_tree(self, node, indent=""):
        if node is None:
            return

        #print(indent + f"I{node.itemset}: {node.support}")
        for child_node in node.children:
            self._print_tree(child_node, indent + "  ")


def build_fp_tree(transactions):
    fp_tree = FPTree()
    for transaction in transactions:
        fp_tree.insert(transaction)
    return fp_tree


def mine_frequent_patterns(fp_tree, min_support, prefix=[]):
    frequent_patterns = []

    for item, nodes in sorted(fp_tree.header_table.items(), key=lambda x: x[1][0].support):
        new_prefix = prefix + [item]

        support = sum(node.support for node in nodes)
        #print(item,support)

        if support >= min_support:
            frequent_patterns.append((new_prefix, support))
            #print(new_prefix,support)

            # Create a conditional pattern base
            conditional_Branch= []
            x=len(nodes)
            for node in nodes:
                #print(item,node.itemset,node.parent.itemset)
                path = []
                curr_node = node.parent
                #print(curr_node.itemset)
                while curr_node.parent is not None:
                    path.append(curr_node.itemset)
                    curr_node = curr_node.parent
                path.reverse()
                for i in range(node.support):
                    conditional_Branch.append(path)
            #print(conditional_pattern_base)
            n = len(conditional_Branch)
            C = {}
            CHT = []

            for x in range(n):
                for i in conditional_Branch[x]:
                    if i in C:
                        C[i] += 1
                    else:
                        C[i] = 1

            for item, count in C.items():
                if count >= min_support:
                    CHT.append(item)

            processed_dataset = []
            for transaction in conditional_Branch:
                p = []
                for i in transaction:
                    if i in CHT:
                        p.append(i)
                processed_dataset.append(p)

            #print(processed_dataset)
            conditional_fp_tree = build_fp_tree(processed_dataset)
            frequent_patterns.extend(mine_frequent_patterns(conditional_fp_tree, min_support, new_prefix))
    #print('\n')
    return frequent_patterns



C = []
path = "/content/drive/MyDrive/Colab Notebooks/Dataset.txt"
fptr = open(path, "r")
transactions = []
with open(path, "r") as fptr:
    for line in fptr:
        transaction = line.split()
        transaction.sort()
        transactions.append(transaction)

n = len(transactions)
x = 0
while x < n:
    token = transactions[x]
    for i in token:
        temp = pattern(i, 1)
        flag = False
        for j in range(0, len(C)):
            if C[j].itemset == temp.itemset:
                C[j].support += 1
                flag = True
                break
        if not flag:
            C.append(temp)
    x += 1

minsup =(70/100)*n
frequent_itemsets=[]
for i in C:
    if(i.support>=minsup):
        frequent_itemsets.append(i)
C1 = sorted(frequent_itemsets, key=lambda x: x.itemset, reverse=False)

C_sorted = sorted(C1, key=lambda x: x.support, reverse=True)

C2 = [pattern_obj.itemset for pattern_obj in C_sorted]
#print(C2)

x1 = 0
processed_dataset = []
while x1 < n:
    p=[]
    for x2 in transactions[x1]:
       if x2 in C2:
           p.append(x2)
    reordered_list = sorted(p, key=lambda x: C2.index(x))
    processed_dataset.append(reordered_list)
    p.clear()
    x1 += 1
fp_tree = build_fp_tree(processed_dataset)

frequent_patterns = mine_frequent_patterns(fp_tree, minsup)

#for pattern, support in frequent_patterns:
   #print(f"Frequent Pattern: {pattern}, Support: {support}")

# Print the FP-tree
fp_tree.print_tree()
end=time.time()
final=end-start
print(final)

Frequent Pattern: ['2'], Support: 7
0.006070375442504883
